In [1]:
val WINDOW = 32;
val STEP = 2;
val SAMPLES = 200000;
val scale = 1/200.0;


In [2]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

In [3]:
// Databricks

//dbutils.fs.mkdirs("/FileStore/fxo/ted")
//dbutils.fs.ls("FileStore")

//val rootDir = "FileStore/fxo/ted/"
//val fileName = "/dbfs/" + rootDir +"a02.dat"



Name: Syntax Error.
Message: 
StackTrace: 

In [4]:
// GCP

val rootDir = "/home/fsainz/src/data/"
val fileName = rootDir +"a02.dat"

In [5]:


//val rawData = sc.textFile("/FileStore/tables/gqh5mjl51507524821196/a02.dat", numPartitions)
var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)

var frames = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]

var pos = 0
var available = 0
val frameEvents = 32 * WINDOW
val frameBytes =  frameEvents * 2 // read shorts 2 bytes..
val nFrames = inFile.length / frameBytes
var dataFrame = Array[Double]()
for (  w  <- 1 to nFrames.toInt ) {
   dataFrame = new Array[Double](frameEvents)
    // new object is need to append at the end!!
 
  for ( i <- 0 until frameEvents) {
   
    val value = input.readShort()   // two bytes
    dataFrame( i ) = (value.toDouble * scale) 
 
    
  }
  frames.append(dataFrame)
   
}
input.close()

In [7]:
val csvFileName = rootDir +"frames-a02.csv"
// FileWriter

val file = new File(csvFileName)
val bw = new BufferedWriter(new FileWriter(file))
var line = ""
for ( f <- frames) {
  f.foreach( x => line +=  x.toString + "," )
  bw.write(line.substring(0,line.length-1) + "\n")
  line = ""
 
}
bw.close()